In [102]:
#Code For Drivers and customer Connection Car Rides                                                   #Created by DVC 19/6/23

import pandas as pd                  #pandas for data tranformation and file handling
from geopy.distance import geodesic  #imported to checkdistance of lat long inputs
import warnings                      # To Ignore warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# For Input of Created one Dataframeusing pandas as  Rides

rides = pd.DataFrame([
    {'ride ID': 1,'pickup_time': 10,                              # by Changing the Pickup time and location We can see the the result being changed Accordingy
     'pickup_location': (40.6892, -74.0445),
     'pickup_address': "Pickup Address 123",                        #currently 3 inputs are availve we can add same types of inputs
     'dropoff_location': (80.6413, -73.7781),
     'dropoff_address': "Dropoff Address 123",                      #usefull in reading the files
    },
    {'ride ID': 2,
     'pickup_time': 50,
     'pickup_location': (55.7590, -73.9845),
     'pickup_address': "Pickup Address 234",
     'dropoff_location': (40.7659, -73.9184),
     'dropoff_address': "Dropoff Address 234",
     },
    {'ride ID': 3,
     'pickup_time': 5,
     'pickup_location': (60.7127, -74.0069),
     'pickup_address': "Pickup Address 345",
     'dropoff_location': (55.6872, -73.9418),
     'dropoff_address': "Dropoff Address 345",
     }
])



# Drvier Dtaframe for input it will update within the code according to booking of driver

drivers = pd.DataFrame([
    {'driver_id': 1, 'price': 15, 'last_dropoff_location': (40.7506, -73.9935)},                       #here price of ride is provided manually
    {'driver_id': 2, 'price': 12, 'last_dropoff_location': (44.7189, 90.005)},
    {'driver_id': 3, 'price': 10, 'last_dropoff_location': (46.6800, -74.0400)},
    {'driver_id': 4, 'price': 50, 'last_dropoff_location': (30.000, 65.000)},
    {'driver_id': 5, 'price': 40, 'last_dropoff_location': (10.001, 90.001)},
    {'driver_id': 6, 'price': 100, 'last_dropoff_location': (70.002, 60.000)},
    {'driver_id': 7, 'price': 150, 'last_dropoff_location': (75.003, -74.0400)},
    {'driver_id': 8, 'price': 200, 'last_dropoff_location': (80.004, 90.004)},
    {'driver_id': 9, 'price': 500, 'last_dropoff_location': (55.005, 90.005)},
    {'driver_id': 10, 'price': 2, 'last_dropoff_location': (60.006, -73.9927)},
    {'driver_id': 11, 'price': 100, 'last_dropoff_location': (70.007, 90.007)},
    {'driver_id': 12, 'price': 90, 'last_dropoff_location': (23.008, -73.9927)},
    {'driver_id': 13, 'price': 80, 'last_dropoff_location': (45.009, 90.009)}])



sorted_rides = rides.sort_values('pickup_time', ascending=True).reset_index(drop=True)           #sorting of avaiable rides according to pickup time in ascending order
#print(sorted_rides)
merged_data = pd.concat([sorted_rides, drivers], axis=1)                                         #merging of driver and rides data

merged_data["pickup_distances"] = 0
merged_data["ride_distances"] = 0
for index, row in merged_data.iterrows():                                                       #for loop iteration for each of the rides with pickup distance , ride distance and Price
    pickup_location = row['pickup_location']
    if pd.isna(pickup_location):                                                                #check for empy cells in data set if empy then exit the loop
        break
    dropoff_location = row['dropoff_location']
    last_dropoff_location = row['last_dropoff_location']
    print("\n\n\n\npickup_location",pickup_location)
    print("\ndropoff_location",dropoff_location)
    try:
      distance = geodesic(pickup_location, last_dropoff_location).miles                       #calcualate the distance from pickup to driver location
      print("\npickup_distance",distance)
    except ValueError:
      #print("Same distance found")
      continue
    merged_data.at[index, "pickup_distances"] = distance
    ride_distance = geodesic(pickup_location, dropoff_location).miles                         #get the distance of total running of vehicle
    print("\ndistance_running",ride_distance)
    merged_data.at[index, "ride_distances"] = ride_distance
    sorted_merged_data = merged_data.sort_values('pickup_distances', ascending=True).reset_index(drop=True)           #soft data in ascendig according to pickup distances

    top_row = sorted_merged_data.dropna().nsmallest(1, ['pickup_distances', 'ride_distances', 'price'])              #get the latest row in dataframe according to pickup distance,ride distance and pricing

    print("\nRide Id :",top_row["ride ID"].reset_index(drop=True)[0],"is assigned to the Driver ID :",top_row["driver_id"].reset_index(drop=True)[0],"for Price :",top_row["price"].reset_index(drop=True)[0],"for Distance :",ride_distance,"Miles" )


    old_pickup_location=top_row["pickup_location"].reset_index(drop=True)[0]                                  #save the output row data in local variables
    temp_dropoff_location=top_row["dropoff_location"] .reset_index(drop=True)[0]
    temp_driver_id = top_row["driver_id"].reset_index(drop=True)[0]
    temp_ride_price = top_row["price"].reset_index(drop=True)[0]
    temp_ride_ID=top_row["ride ID"].reset_index(drop=True)[0]                                                 #save the output row data in local variables


    drop_booked_row2 = merged_data[merged_data['ride ID'] == temp_ride_ID].index                             #mark the location of booked ride in input dataframe
    merged_data = merged_data.drop(drop_booked_row2)                                                         #drop the row which is booked

    drop_booked_row = drivers[drivers['driver_id'] == temp_driver_id].index                                  #mark the location of booked ride in main driver dataframe


    new_row = pd.Series({'driver_id': temp_driver_id,'price': temp_ride_price,'last_dropoff_location':temp_dropoff_location})    #add new row in driver dataframe for booked driver with updatedl ocation
    print("\nFor Driver",top_row["driver_id"].reset_index(drop=True)[0],"New_pickup_location:",temp_dropoff_location)



    old_driver_location = drivers.loc[drop_booked_row, 'last_dropoff_location']                                   #extract the location of booked ride linked with dropout location
    print("\nOld location of Driver :",old_driver_location.reset_index(drop=True)[0])
    drivers = drivers.drop(drop_booked_row)                                                                       #remove the driver entry so that same driver entry will not be linked with other ride

    drivers=drivers.append(new_row, ignore_index=True)                                                            #add new driver details for booked driver

    drop_booked_row = drivers[drivers['driver_id'] == temp_driver_id].index                                       #point the drata frame location to add new location of driver
    new_driver_location = drivers.loc[drop_booked_row, 'last_dropoff_location']                                   #add the new location
    print("Updated Driver Location :",new_driver_location.reset_index(drop=True)[0])
    drop_booked_row1 = sorted_merged_data[sorted_merged_data['ride ID'] == temp_ride_ID].index                    #remove booked row from sorted dataset
    sorted_merged_data = sorted_merged_data.drop(drop_booked_row1)


    drop_booked_row3 = rides[rides['ride ID'] == temp_ride_ID].index                                              #point the location of booked ride in ride dataframe
    rides = rides.drop(drop_booked_row3)                                                                          #remove the booked ride entry from dataframe











pickup_location (60.7127, -74.0069)

dropoff_location (55.6872, -73.9418)

pickup_distance 1379.819227693523

distance_running 347.8171784503888

Ride Id : 2.0 is assigned to the Driver ID : 3 for Price : 10 for Distance : 347.8171784503888 Miles

For Driver 3 New_pickup_location: (40.7659, -73.9184)

Old location of Driver : (46.68, -74.04)
Updated Driver Location : (40.7659, -73.9184)




pickup_location (40.6892, -74.0445)

dropoff_location (80.6413, -73.7781)

pickup_distance 6469.394665880579

distance_running 2765.538857802896

Ride Id : 3.0 is assigned to the Driver ID : 1 for Price : 15 for Distance : 2765.538857802896 Miles

For Driver 1 New_pickup_location: (55.6872, -73.9418)

Old location of Driver : (40.7506, -73.9935)
Updated Driver Location : (55.6872, -73.9418)




pickup_location (55.759, -73.9845)

dropoff_location (40.7659, -73.9184)

pickup_distance 627.624781422383

distance_running 1035.925871020187

Ride Id : 1.0 is assigned to the Driver ID : 2.0 for Price :